In [ ]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect('/home/greenman225/Code/github/donkey_golf/donkey_golf/site.db')
c = conn.cursor()

# Show all tables in our database 

In [ ]:
pd.read_sql('SELECT name FROM sqlite_master WHERE type = "table"', conn)

In [ ]:
pd.read_sql('SELECT * from user', conn)